Author: Will Blanton

# Imports

In [161]:
import pandas as pd
import torch
import torch.nn as nn
import random
import itertools

import torch_geometric
from sentence_transformers import SentenceTransformer
from torch.utils.data import Dataset
from torch.nn.functional import cosine_similarity
from torch_geometric.data import Data

# Helper Functions

# Create Dataset

In [32]:
print(f"Cuda available: {torch.cuda.is_available()}")

Cuda available: True


In [34]:
model = SentenceTransformer('all-MiniLM-L6-v2')

D:\PyCharmProjects\nyt-connections-solver\.venv\Lib\site-packages\huggingface_hub\file_download.py:144: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Will Blanton\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

In [98]:
import ast
example_df = pd.read_csv('data/connections.csv', index_col=0)
example_df.drop(columns='category', inplace=True)

# fix issues with incorrect format... (not worth going in and adding a mechanism to correct...)
example_df.loc[1298, "connections"] = "['line', 'plane', 'point', 'solid']"
example_df.loc[1892, "connections"] = "['abyss', 'fly', 'matrix', 'thing']"

# remove april fools samples since they include emojis or other potentially noisy samples
example_df = example_df[~example_df['date'].str.contains("04-01")]
example_df.reset_index(drop=True, inplace=True)

example_df['connections'] = example_df['connections'].apply(ast.literal_eval)
example_df

,date,connections
0,2023-06-12,"[kayak, level, mom, race car]"
1,2023-06-12,"[option, return, shift, tab]"
2,2023-06-12,"[bucks, heat, jazz, nets]"
3,2023-06-12,"[hail, rain, sleet, snow]"
4,2023-06-13,"[are, queue, sea, why]"
...,...,...
2751,2025-05-01,"[pot, prize, purse, reward]"
2752,2025-05-02,"[bottle, break, goose, turtle]"
2753,2025-05-02,"[dog, link, rib, wing]"
2754,2025-05-02,"[brace, post, prop, support]"


In [99]:
example_df.groupby(by='date').agg(
    {
        "connections": lambda x: list(itertools.chain.from_iterable(x)) 
    }
)

,connections
date,
2023-06-12,"[kayak, level, mom, race car, option, return, ..."
2023-06-13,"[are, queue, sea, why, essence, people, time, ..."
2023-06-14,"[amigo, king, stooge, tenor, lab, peke, pit, p..."
2023-06-15,"[bat, iron, spider, super, dust, mop, sweep, v..."
2023-06-16,"[green, mustard, plum, scarlet, blue, down, gl..."
...,...
2025-04-28,"[bore, drain, exhaust, tire, fiber, fingerprin..."
2025-04-29,"[gemstone, infield, rhombus, suit, ladder, mou..."
2025-04-30,"[dynasty, engross, gimmick, mildew, face, imag..."


In [100]:
example_df[example_df['date'] == '2023-06-21'].reset_index(drop=True)

,date,connections
0,2023-06-21,"[fish, goat, scales, twins]"
1,2023-06-21,"[crane, jay, swallow, turkey]"
2,2023-06-21,"[chad, georgia, jordan, togo]"
3,2023-06-21,"[date, kiwi, lemon, orange]"


In [148]:
class ConnectionsData(Data):
    # needed to adjust the edge group indices for batching
    def __inc__(self, key, value, *args, **kwargs):
        if key == 'group_indices':
            # group_indices refers to node indices → so shift by number of nodes
            return self.x.size(0)
        else:
            return super().__inc__(key, value, *args, **kwargs)

In [149]:
"""

TODO: add more features to handle complicated cases
- phonetic word embeddings
- character-level embeddings (either trained via RNN or pre-trained)
- n-gram?
"""
class ConnectionsGraphDataset(Dataset):
    def __init__(
        self,
        puzzle_df: pd.DataFrame,
        word_emb_model: SentenceTransformer,
        negative_ratio: int = 3,
        include_purple: bool = False
    ):
        super().__init__()
        self.puzzle_df = puzzle_df
        self.negative_ratio = negative_ratio
        self.include_purple = include_purple
        self.data_list = []

        # get lists of words for each puzzle 
        words_per_date = (
            self.puzzle_df
            .groupby('date')['connections']
            .agg(lambda lists: list(itertools.chain.from_iterable(lists)))
        )

        # create data object for each puzzle
        for date, word_list in words_per_date.items():
            if len(word_list) != 16:
                raise ValueError(f'Word list length {len(word_list)} does not match 16 words')
            data = self._build_single_graph(date, word_list, word_emb_model)
            self.data_list.append(data)

    def _build_single_graph(self, date, word_list, word_emb_model):
        
        # create node features 
        x = word_emb_model.encode(word_list, convert_to_tensor=True)
        word2idx = {w: i for i, w in enumerate(word_list)}

        edge_index, edge_attr = self._make_fully_connected(x)

        positives = self._collect_positives(date, word2idx)
        negatives = self._sample_negatives(len(word_list), positives)

        # remove purple pairs (already handled for negatives)
        if not self.include_purple:
            positives = positives[:-1]

        group_indices = torch.tensor(positives + negatives, dtype=torch.long)
        group_labels = torch.tensor(
            [1]*len(positives) + [0]*len(negatives),
            dtype=torch.float
        )
        
        data = ConnectionsData(
            x=x,    # (16, embed_dim)
            edge_index=edge_index,  # (2, num_edges)
            edge_attr=edge_attr, # (num_edges, 1)
            group_indices=group_indices,  # (num_groups, 4)
            group_labels=group_labels     # (num_groups,)
        )

        data.word_list = word_list
        return data

    def _make_fully_connected(self, x):
        num_nodes = x.size(0)
        # all i<j pairs
        pairs = list(itertools.combinations(range(num_nodes), 2))
        edge_index = torch.tensor(pairs, dtype=torch.long).t().contiguous()
        
        # add reverse direction
        edge_index = torch.cat([edge_index, edge_index.flip(0)], dim=1)

        # compute cosine-similarity edge weights
        src = x[edge_index[0]]
        dst = x[edge_index[1]]
        edge_attr = cosine_similarity(src, dst).unsqueeze(1)

        return edge_index, edge_attr

    def _collect_positives(self, date, word2idx):
        subset = self.puzzle_df[self.puzzle_df['date'] == date]
        positives = []
        for _, row in subset.iterrows():
            idxs = [word2idx[w] for w in row['connections']]
            positives.append(sorted(idxs))
        return positives

    def _sample_negatives(self, num_nodes, positives):
        positive_set = set(tuple(g) for g in positives)
        all_quads = list(itertools.combinations(range(num_nodes), 4))
        random.shuffle(all_quads)

        negatives = []
       
        if self.include_purple: 
            max_neg = self.negative_ratio * len(positives)
        else:
            max_neg = self.negative_ratio * (len(positives) - 1)
        
        for quad in all_quads:
            if len(negatives) >= max_neg:
                break
            if tuple(quad) not in positive_set:
                negatives.append(list(quad))
        return negatives

    def __len__(self):
        return len(self.data_list)

    def __getitem__(self, idx):
        return self.data_list[idx]    

In [150]:
example_df['date'].nunique()

689

In [151]:
dataset = ConnectionsGraphDataset(example_df, model, negative_ratio=3, include_purple=False)
data = next(iter(dataset))
data

ConnectionsData(x=[16, 384], edge_index=[2, 240], edge_attr=[240, 1], group_indices=[12, 4], group_labels=[12], word_list=[16])

In [152]:
x = data.x
x

tensor([[ 0.0211,  0.0576,  0.0229,  ..., -0.0708, -0.0298,  0.0303],
        [-0.0815, -0.0077, -0.0857,  ...,  0.0268, -0.0626, -0.0200],
        [-0.0583,  0.0731,  0.0069,  ..., -0.0102,  0.0923,  0.0631],
        ...,
        [-0.0259,  0.0191,  0.0624,  ..., -0.0290, -0.0521,  0.0364],
        [-0.0522,  0.0074, -0.0133,  ..., -0.0651, -0.0631,  0.0272],
        [-0.0633,  0.0449,  0.0684,  ..., -0.0407, -0.0955,  0.0191]],
       device='cuda:0')

In [153]:
group_embeds = x[data.group_indices]
group_embeds.view(group_embeds.size(0), -1).shape

torch.Size([12, 1536])

In [154]:
data.group_labels.shape

torch.Size([12])

In [159]:
from torch_geometric.loader import DataLoader
for batch in DataLoader(dataset, batch_size=2, shuffle=True):
    print(batch)
    print(batch.group_indices)

    x = batch.x
    group_embeds = x[batch.group_indices]
    print(group_embeds.view(group_embeds.size(0), -1).shape)
    
    break

ConnectionsDataBatch(x=[32, 384], edge_index=[2, 480], edge_attr=[480, 1], group_indices=[24, 4], group_labels=[24], word_list=[2], batch=[32], ptr=[3])
tensor([[ 0,  1,  2,  3],
        [ 4,  5,  6,  7],
        [ 8,  9, 10, 11],
        [ 2,  4,  5, 12],
        [ 4,  5, 11, 13],
        [ 1,  2, 13, 14],
        [ 1,  3,  8,  9],
        [ 2,  8,  9, 13],
        [ 5,  8, 10, 13],
        [ 0,  2,  5, 12],
        [ 7,  8, 10, 15],
        [ 5, 11, 12, 15],
        [16, 17, 18, 19],
        [20, 21, 22, 23],
        [24, 25, 26, 27],
        [17, 24, 28, 30],
        [16, 19, 29, 31],
        [19, 23, 27, 28],
        [17, 18, 26, 28],
        [16, 18, 23, 30],
        [22, 24, 28, 31],
        [17, 19, 25, 30],
        [17, 22, 23, 28],
        [16, 19, 23, 26]])
torch.Size([24, 1536])


# Model Architecture

In [162]:
class GraphEncoderLayer(nn.Module):
    def __init__(self, hidden_dim, attn_heads, dropout, edge_dim):
        super().__init__()
        self.conv = torch_geometric.nn.TransformerConv(
            in_channels=hidden_dim,
            out_channels=hidden_dim,
            heads=attn_heads,
            dropout=dropout,
            edge_dim=edge_dim,
            beta=True
        )
         
        self.norm = nn.LayerNorm(hidden_dim)

    def forward(self, x, edge_index, edge_attr):
        residual = x
        x = self.conv(x, edge_index, edge_attr)
        x = self.norm(x + residual)  # Residual + Norm
        return x


In [ ]:
class ConnectionsGNN(torch.nn.Module):
    def __init__(
            self,
            in_dim,
            num_layers=1,
            hidden_size=256,
            output_size=1,
            attn_heads=8,
            dropout=0.2,
            edge_dim=1
        ):
        super().__init__()

        self.lin_proj = nn.Linear(in_dim, hidden_size)
        
        self.layers = torch.nn.ModuleList()
        
        for _ in range(num_layers):
            self.layers.append(GraphEncoderLayer(hidden_size, attn_heads, dropout, edge_dim))
            
        self.scorer = torch.nn.Sequential(
            nn.Linear(hidden_size * 4, hidden_size),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_size, hidden_size),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_size, output_size)
        )
        
    def forward(self, x, edge_index, edge_attr, group_indices):
        
        x = self.lin_proj(x)    # (num_nodes, in_dim) -> (num_nodes, hidden_dim)

        # compute graph embeddings 
        for layer in self.layers:
            x = layer(x, edge_index, edge_attr) # (num_nodes, hidden_dim)
            
        # concat groups for final scores
        group_embs = x[group_indices]   # (num_groups, 4, hidden_dim)
        group_embs = group_embs.view(group_embs.size(0), -1)    # (num_groups, 4 * hidden_dim)
        
        logits = self.scorer(group_embs)    # (num_groups, 1)
        return logits.squeeze(-1)   # (num_groups,)

# Train Model